#Multiple Indexing 

In [ ]:
import utils
import pandas as pd

filenames = ['data/measured_real_power.csv',
             'data/measured_reactive_power.csv']

df = utils.read_dir(filenames)
df.shape

In [ ]:
sorted(df.columns, key=lambda x: x[-2:])[:4]

What if we want all the `mesured_real_power` or all attributes of `triplex_meter_5'?

- could use `startswith()` or `endswith()`.
- could use a regular expression with `re`.
- **multi-index** or **stack**

## Multi-Index 

In [ ]:
tuples = [(x.split(':')[0], x.split(':')[1]) for x in df.columns]
tuples[:3]

In [ ]:
col_index = pd.MultiIndex.from_tuples(tuples, names=['attr','node'])

multi = df.copy()
multi.columns = col_index
multi.head()

In [ ]:
multi = multi.swaplevel(0,1,axis=1)
multi.head()

What is the total power used by each `triplex_meter`?

In [ ]:
multi.groupby(level=0, axis=1).sum().sum().order().plot(kind='barh')

## Stacking and Multi-indexing 

In [ ]:
stack = df.unstack()
stack.head()

In [ ]:
stack = stack.reset_index(level=0)
stack.head()

In [ ]:
stack.columns = ['attr:node', 'value']
stack.head()

In [ ]:
stack['attr'] = stack['attr:node'].apply(lambda x: x.split(':')[0])
stack['node'] = stack['attr:node'].apply(lambda x: x.split(':')[1])
stack = stack[['node', 'attr', 'value']]
stack.head()

### Query

In [ ]:
stack[stack.node=='triplex_meter_0'].head()

In [ ]:
((stack.node=='triplex_meter_0') & (stack.attr == 'measured_real_power')).head()

In [ ]:
tmp = stack[(stack.node=='triplex_meter_0') & (stack.attr == 'measured_real_power')]

print(tmp['node'].value_counts())
print(tmp['attr'].value_counts())

###Stack to multi-index 

In [ ]:
stack = stack.reset_index().set_index(['index','node','attr'])

In [ ]:
stack.head()

In [ ]:
stack.unstack(level=2).head()

In [ ]:
stack.unstack(level=1).head()

In [ ]:
stack.unstack(level=[1,2]).head()

In [ ]:
tmp = stack.unstack(level=[1,2])
tmp['value']['triplex_meter_0'].head()

In [ ]:
tmp.columns.droplevel(0)

In [ ]:
tmp.columns = tmp.columns.droplevel(0)
tmp.head()

In [ ]:
tmp['triplex_meter_0'].head()

### Yet another way

In [ ]:
stack.head()

In [ ]:
stack.unstack(['node','attr'])['value'].head()

## Adding a row to a multi-index 

In [ ]:
multi.head()

In [ ]:
stack.head()

What are some possibilities?  What's the fastest?

In [ ]:
import math

def compute_mag(df):
    total =  (df['measured_real_power']*df['measured_real_power'] + 
              df['measured_reactive_power']*df['measured_reactive_power'])
    
    return total.apply(math.sqrt)

In [ ]:
tmp = stack.unstack('attr')['value']
tmp.head()

In [ ]:
total = tmp['measured_real_power']*tmp['measured_real_power'] + tmp['measured_reactive_power']*tmp['measured_reactive_power']
tmp['mag'] = total.apply(math.sqrt)
tmp.head()

In [ ]:
tmp.unstack(level=1).head()

In [ ]:
tmp.unstack('node').head()

In [ ]:
tmp.unstack('node')['mag'].sum(axis=1).plot(legend=False)